### Description

Given two or more flexibility experiments, generates tables comparing the aggregate flexibility measures (worst, average, and best learning and adaption costs).

### Setup

In [ ]:
import csv, os
import numpy as np
import pandas as pd

from scipy import stats
import scikit_posthocs as sp

# Path where to store anlysis results.
save_path = '../../../flexbench-data/3_NSGA-II_varying_goals/comp_baseline_varying_goals_active_inactive/'

# Path to experimental data (each directory should contain a 'logs' subdirectory).
# Baseline refers to where the data for learning from scratch is.
# Adaption refers to where the data for adaption is.
baseline_paths = {
    'Baseline (from scratch)': {
        'path' : '../../../flexbench-data/2_baseline_NSGA-II/2-2_tournament/',
        'popSize': 100,
        'nGens': 50,
    },
}

adaption_paths = {
    'Baseline (adaption)' : {
        'path' : '../../../flexbench-data/2_baseline_NSGA-II/2-2_tournament/',
        'path_target': '../../../flexbench-data/2_baseline_NSGA-II/2-2_tournament/',
        'popSize': 100,
        'nGens': 50,
    },
    'Varying Goals (adaption)' : {
        'path' : '../../../flexbench-data/3_NSGA-II_varying_goals/3-1_varying_goals/',
        'path_target': '../../../flexbench-data/2_baseline_NSGA-II/2-2_tournament/',
        'popSize': 100,
        'nGens': 50,
    },
    'Active-Inactive (adaption)' : {
        'path' : '../../../flexbench-data/3_NSGA-II_varying_goals/3-2_varying_goals_active_inactive/',
        'path_target': '../../../flexbench-data/2_baseline_NSGA-II/2-2_tournament/',
        'popSize': 100,
        'nGens': 50,
    },
}

# Target tasks. Adaption are identified later for each directory.
tasks_target = ["steel", "tungsten_alloy", "steel_dummy", "inconel_718"]

# Number of repetitions of experiments.
repetitions = 100

# Value for calculating computational effort (percentage of certainty).
z = 0.99

# If True, additionally calculates computational effort and evaluations
# based on a percentage of the target hypervolume.
relaxed_include = True
relaxed_percentage = 0.99

### Functions for loading data and processing cost data.

In [ ]:
def load_evals_relaxed(
    path_hypervolume_target,
    path_hypervolume_current,
    repetitions,
    nGens,
    popSize,
    relaxed_percentage,
):
    """
    Return an array with the number of evaluations needed for finding a percentage
    of target hypervolumes.
    """ 
    print("Loading %s..." % path_hypervolume_current)
    
    with open(path_hypervolume_target, "r", newline="") as f:
        reader = csv.reader(f, delimiter=",")
        hypervolume_target = np.array([max([float(x) for x in row]) for row in reader]) * relaxed_percentage
        hypervolume_target = hypervolume_target[:repetitions]
    
    with open(path_hypervolume_current, "r", newline="") as f:
        reader = csv.reader(f, delimiter=",")
        data = [[float(x) for x in row] for row in reader]
        data = data[:repetitions]
    
    evals_relaxed = [-1 for _ in range(len(data))]
    for idx in range(len(data)):
        for gen in range(nGens+1):
            if data[idx][gen] >= hypervolume_target[idx]:
                evals_relaxed[idx] = (gen+1)*popSize
                break
    
    return np.array(evals_relaxed)
    
def calc_min_effort(evals, nRuns, nGens, popSize, z):
    '''Calculates the minimum number of evaluations for suceeding with probability z.'''

    #Initialize generations vector.
    gensVec = [0 for i in range(nGens+1)]
    
    #Fill generations vector by iterating evaluations vector.
    for x in evals :
        if x != -1 :
            idx = int(x/popSize) - 1
            gensVec[idx] = gensVec[idx] + 1
    
    #Calculate cumulative vector.
    for i in range(1,len(gensVec)) :
        gensVec[i] = gensVec[i] + gensVec[i-1]
    
    #Minimum effort initially infinite.
    minEff = float('+Inf')
    
    #For each generation.
    for i in range(len(gensVec)) :
    
        #Calculate probability based on frequencies.
        prob = gensVec[i]/float(nRuns)
        
        if prob == 0:
            prob = 1e-7
        
        #Calculate effort.
        if prob == 1.0 :
            r = 1.0
        else :
            r = np.ceil(np.log(1-z)/np.log(1-prob))
            
        currEff = popSize * (i+1) * r
        
        #Update minimum effort.
        if currEff < minEff :
            minEff = currEff
        
    #Return minimum effort.
    return minEff

def gen_data_dict(baseline_paths, adaption_paths, tasks_target, repetitions, z, relaxed_percentage):
    '''Returns a dictionary with baseline and adaption experiments as keys and CE as values.'''

    # Initialize data dict.
    data_dict = {}
    
    # For each baseline experiment.
    for exp in baseline_paths.keys():
    
        # Initilialize entry.
        data_dict[exp] = {}
        
        # For each target.
        for target in tasks_target:
        
            # Read evals, calculate and store CE.
            evals = load_evals_relaxed(
                baseline_paths[exp]['path'] + 'logs/' + target + '/training/hypervolume.csv',
                baseline_paths[exp]['path'] + 'logs/' + target + '/training/hypervolume.csv',
                repetitions,
                baseline_paths[exp]['nGens'],
                baseline_paths[exp]['popSize'],
                relaxed_percentage,
            )
            CE = calc_min_effort(
                evals,
                repetitions,
                baseline_paths[exp]['nGens'],
                baseline_paths[exp]['popSize'],
                z,
            )
            data_dict[exp][target] = CE
    
    # For each adaption experiment.
    for exp in adaption_paths.keys():
    
        # Initialize entry.
        data_dict[exp] = {}
        
        # For each target.
        for target in tasks_target:
        
            # Identify pairs of source to target.
            pairs = [
                pair
                for pair in os.listdir(adaption_paths[exp]['path'] + 'logs/')
                if ('_to_' in pair) and (pair.split('_to_')[-1] == target)
            ]
            
            # For each pair.
            for pair in pairs:
            
                # Read evals, calculate and store CE.
                evals = load_evals_relaxed(
                    adaption_paths[exp]['path_target'] + 'logs/' + target + '/training/hypervolume.csv',
                    adaption_paths[exp]['path'] + 'logs/' + pair + '/adaption/hypervolume.csv',
                    repetitions,
                    adaption_paths[exp]['nGens'],
                    adaption_paths[exp]['popSize'],
                    relaxed_percentage,
                )
                CE = calc_min_effort(
                    evals,
                    repetitions,
                    adaption_paths[exp]['nGens'],
                    adaption_paths[exp]['popSize'],
                    z,
                )
                data_dict[exp][pair] = CE
                
    # Return data dict.
    return data_dict

### Functions for generating comparative table of aggregated measures.

In [ ]:
def save_table(table, path):
    print("Saving %s..." % path)
    with open(path, "w", newline="") as f:
        writer = csv.writer(f, delimiter=",")
        writer.writerows(table)

def gen_aggregated_table(data_dict, save_path):
    '''Given a data dict, generates a table with worst, average, and best cases.'''
    
    # Initialize table.
    table = [['Algorithm', 'Worst Case', 'Average Case', 'Best Case']]
    
    # For each experiment, calculate measures and add row to table.
    for exp in data_dict.keys():
        values = [data_dict[exp][task] for task in data_dict[exp].keys()]
        worst_case = max(values)
        avg_case = np.mean(values)
        best_case = min(values)
        table.append([exp, worst_case, avg_case, best_case])
    
    # Save table.
    save_table(table, save_path + 'table_aggregated_values.csv')

### Functions for generating table with Friedmann and post-hoc Nemeyi statistics.

In [ ]:
def perform_statistical_tests(data_dict, save_path):
    
    # Format data.
    table_values = [
        [data_dict[exp][task] for task in data_dict[exp].keys()]
        for exp in data_dict.keys()
    ]
    
    # Fix shorter rows.
    # Assumes adaption rows have more values because of different source x target pairs.
    # Assumes adaption rows have all the same length, with equal number of values per target.
    len_baseline = min([len(row) for row in table_values])
    len_adaption = max([len(row) for row in table_values])
    n_per_target = int(len_adaption/len_baseline)
    for idx in range(len(table_values)):
        if len(table_values[idx]) < len_adaption:
            table_values[idx] = np.repeat(table_values[idx], n_per_target)
    
    # Run Friedman test.
    result_friedman = stats.friedmanchisquare(*table_values)
    
    # Generate mean rankings.
    result_ranks = np.mean(stats.rankdata(table_values, axis=0), axis=1)
    
    # Run Nemenyi test.
    result_nemenyi = sp.posthoc_nemenyi_friedman(np.array(table_values).T)
    
    # Build tables.
    table_friedman = [
        ['Friedman Statistic', result_friedman.statistic],
        ['Friedman p-value', result_friedman.pvalue],
    ]
    
    table_mean_rankings = [
        [exp for exp in data_dict.keys()],
        [rank for rank in result_ranks],
    ]
    
    table_nemenyi = pd.DataFrame(
        result_nemenyi.values, 
        index = [exp for exp in data_dict.keys()],
        columns = [exp for exp in data_dict.keys()],
    )
    
    # Save tables.
    save_table(table_friedman, save_path + 'table_friedman.csv')
    save_table(table_mean_rankings, save_path + 'table_mean_rankings.csv')
    
    print("Saving %s..." % save_path + 'table_nemenyi.csv')
    table_nemenyi.to_csv(save_path + 'table_nemenyi.csv', sep=',')

### Runs complete analysis.

In [ ]:
# Create main analysis directory.
try:
    os.mkdir(save_path)
except FileExistsError:
    pass

# Subdirectory without relaxed target.
try:
    os.mkdir(save_path + 'analysis/')
except FileExistsError:
    pass

# Build data dict.
data_dict = gen_data_dict(baseline_paths, adaption_paths, tasks_target, repetitions, z, 1.0)

# Generate and save tables with aggretated measures.
gen_aggregated_table(data_dict, save_path + 'analysis/')

# Generate and save tables with statistical tests.
perform_statistical_tests(data_dict, save_path + 'analysis/')

# If including relaxed target.
if relaxed_include:

    # Subdirectory with relaxed target.
    try:
        os.mkdir(save_path + 'analysis_relaxed_target_%.2f/' % relaxed_percentage)
    except FileExistsError:
        pass
    
    # Build data dict.
    data_dict = gen_data_dict(baseline_paths, adaption_paths, tasks_target, repetitions, z, relaxed_percentage)

    # Generate and save tables with aggretated measures.
    gen_aggregated_table(data_dict, save_path + 'analysis_relaxed_target_%.2f/' % relaxed_percentage)

    # Generate and save tables with statistical tests.
    perform_statistical_tests(data_dict, save_path + 'analysis_relaxed_target_%.2f/' % relaxed_percentage)